<h3><b>Activity 1: Implementing RSA</b></h3>

<h4><b>RSA Key Generation, Encryption, and Decryption</b></h4>

In [1]:
from Crypto.Util.number import inverse, getPrime
import random

# Step 1: Key Generation
def generate_rsa_keys():
    # Choosing two small primes manually or randomly
    p = 61  # example prime
    q = 53  # example prime
    n = p * q
    phi = (p - 1) * (q - 1)

    e = 17  # common choice
    d = inverse(e, phi)  # Modular inverse of e modulo phi

    public_key = (e, n)
    private_key = (d, n)

    return public_key, private_key, p, q

# Step 2: Encrypt a message
def encrypt(message, public_key):
    e, n = public_key
    cipher = pow(message, e, n)
    return cipher

# Step 3: Decrypt the message
def decrypt(ciphertext, private_key):
    d, n = private_key
    message = pow(ciphertext, d, n)
    return message

# Testing
public_key, private_key, p, q = generate_rsa_keys()

print(f"Public Key (e, n): {public_key}")
print(f"Private Key (d, n): {private_key}")
print(f"p: {p}, q: {q}")

message = 5  # Example message
ciphertext = encrypt(message, public_key)
print(f"Encrypted Message: {ciphertext}")

decrypted_message = decrypt(ciphertext, private_key)
print(f"Decrypted Message: {decrypted_message}")


Public Key (e, n): (17, 3233)
Private Key (d, n): (2753, 3233)
p: 61, q: 53
Encrypted Message: 3086
Decrypted Message: 5


<h3><b>Activity 2: Simulating Diffie-Hellman Key Exchange</b></h3>

In [2]:
import random

# Public values (agreed upon)
p = 23   # A small prime number
g = 5    # A primitive root modulo 23

# Each student selects a private key
private_key_student1 = random.randint(1, p-1)
private_key_student2 = random.randint(1, p-1)

# Public keys
A = pow(g, private_key_student1, p)
B = pow(g, private_key_student2, p)

print(f"Student 1's public key (A): {A}")
print(f"Student 2's public key (B): {B}")

# Exchange A and B, then compute the shared secret
shared_secret_student1 = pow(B, private_key_student1, p)
shared_secret_student2 = pow(A, private_key_student2, p)

print(f"Shared Secret computed by Student 1: {shared_secret_student1}")
print(f"Shared Secret computed by Student 2: {shared_secret_student2}")

# Should be the same!
assert shared_secret_student1 == shared_secret_student2


Student 1's public key (A): 20
Student 2's public key (B): 6
Shared Secret computed by Student 1: 2
Shared Secret computed by Student 2: 2


<h3><b>Optional Challenge: Hybrid System (Diffie-Hellman + AES)</b></h3>

In [3]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib

# Using the shared secret from Diffie-Hellman
shared_secret = shared_secret_student1

# Derive AES key from shared secret (must be 16 bytes for AES-128)
key = hashlib.sha256(str(shared_secret).encode()).digest()[:16]

# Encrypt a message
def encrypt_message(message, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), AES.block_size))
    iv = cipher.iv
    return iv + ct_bytes

# Decrypt a message
def decrypt_message(encrypted_data, key):
    iv = encrypted_data[:16]
    ct = encrypted_data[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode()

# Example Usage
message = "Hello Partner!"
encrypted = encrypt_message(message, key)
print(f"Encrypted Message: {encrypted.hex()}")

decrypted = decrypt_message(encrypted, key)
print(f"Decrypted Message: {decrypted}")


Encrypted Message: 2bd7d06eb47d887468207c62118fd641206f2828b6e4e665917c9e4baadf87a6
Decrypted Message: Hello Partner!
